In [50]:
import os
import requests
from google.colab import userdata

# Access the API key
news_api_key = userdata.get("NEWS_API_KEY")

def fetch_news(topic="technology", language="en", page_size=5):
    """
    Fetches the latest news articles from NewsAPI based on topic and language, limited to a specific number of articles.
    
    Parameters:
        topic (str): Category of news to fetch (default is "technology").
        language (str): Language code for the articles (default is "en" for English).
        page_size (int): Number of articles to fetch (default is 5).
        
    Returns:
        list: A list of dictionaries, each containing details of a news article.
    """
    url = "https://newsapi.org/v2/top-headlines"
    params = {
        "apiKey": news_api_key,
        "category": topic,
        "language": language,
        "pageSize": page_size  # Limit number of articles
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        data = response.json()
        articles = data.get("articles", [])
        
        formatted_articles = [
            {
                "title": article["title"],
                "description": article["description"],
                "url": article["url"],
                "publishedAt": article["publishedAt"]
            }
            for article in articles
        ]
        return formatted_articles
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
        return []
    except ValueError as e:
        print("Error processing JSON response:", e)
        return []

In [51]:
# Load Hugging Face API key
huggingface_api_key = userdata.get("HUGGINGFACE_API_KEY")

def summarize_text_with_huggingface(text):
    """
    Summarizes the given text using Hugging Face's Inference API for a summarization model.
    
    Parameters:
        text (str): The text to be summarized.
        
    Returns:
        str: A summarized version of the input text.
    """
    # Define the Hugging Face model endpoint for summarization
    api_url = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
    headers = {"Authorization": f"Bearer {huggingface_api_key}"}
    payload = {"inputs": text}
    
    try:
        # Make the API request
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()
        
        # Extract the summary from the response
        summary = response.json()[0]["summary_text"]
        return summary
    except requests.exceptions.RequestException as e:
        print(f"Error during summarization: {e}")
        return ""

In [52]:
def fetch_and_summarize_news(topic="technology", language="en"):
    """
    Fetches the latest news articles on a given topic and provides summaries.
    
    Parameters:
        topic (str): Category of news to fetch (default is "technology").
        language (str): Language code for the articles (default is "en" for English).
        
    Returns:
        list: A list of dictionaries, each containing the title, summary, and link to a news article.
    """
    # Fetch the latest news articles
    articles = fetch_news(topic=topic, language=language)
    
    summarized_articles = []
    
    # Summarize each article's content
    for article in articles:
        title = article['title']
        description = article['description']
        url = article['url']
        
        # Combine title and description for a more comprehensive summary
        if description:
            text_to_summarize = f"{title}. {description}"
        else:
            text_to_summarize = title  # Use title alone if description is missing
        
        # Get summary
        summary = summarize_text_with_huggingface(text_to_summarize)
        
        # Store summarized article
        summarized_articles.append({
            "title": title,
            "summary": summary,
            "url": url
        })
    
    return summarized_articles

In [53]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Create dropdown for news category selection
category_dropdown = widgets.Dropdown(
    options={
        'Technology': 'technology',
        'Sports': 'sports',
        'Business': 'business',
        'Entertainment': 'entertainment'
    },
    value='technology',  # default value
    description='Category:',
)

# Create the button and output display
fetch_button = widgets.Button(description="Fetch Headlines")
output = widgets.Output()

def display_summaries(summarized_articles):
    """
    Displays a list of summarized articles in a card-like format using HTML for a more user-friendly layout.
    
    Parameters:
        summarized_articles (list): List of dictionaries containing title, summary, and URL for each article.
    """
    html_content = ""
    
    for article in summarized_articles:
        title = article["title"]
        summary = article["summary"]
        url = article["url"]
        
        # Create a card-like HTML structure for each article
        html_content += f"""
        <div style="
            border: 1px solid #ddd; 
            border-radius: 8px; 
            padding: 16px; 
            margin: 10px 0; 
            background-color: #f9f9f9;">
            <h3 style="margin: 0;"><a href="{url}" target="_blank" style="text-decoration: none; color: #0073e6;">{title}</a></h3>
            <p style="font-size: 14px; color: #333;">{summary}</p>
            <a href="{url}" target="_blank" style="color: #0073e6; font-weight: bold;">Read more</a>
        </div>
        """
    
    # Display the final HTML content
    display(HTML(html_content))

def on_button_click(b):
    # Clear previous output
    output.clear_output()
    
    # Fetch selected category
    selected_category = category_dropdown.value
    
    # Fetch and display news summaries when button is clicked
    with output:
        print(f"Fetching and summarizing articles for category: {selected_category.capitalize()}...")
        news_summaries = fetch_and_summarize_news(topic=selected_category)
        
        # Display the summarized articles using the new card-like display
        display_summaries(news_summaries)

# Link button click event to the function
fetch_button.on_click(on_button_click)

# Display the dropdown, button, and output area
display(category_dropdown)
display(fetch_button)
display(output)

Dropdown(description='Category:', options={'Technology': 'technology', 'Sports': 'sports', 'Business': 'busine…

Button(description='Fetch and Summarize Top 5 Headlines', style=ButtonStyle())

Output()